In [10]:
#referenced https://www.freecodecamp.org/news/how-to-create-a-gui-quiz-application-using-tkinter-and-open-trivia-db/


import random
from tkinter import *
import requests

url = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
get_request = requests.get(url).json()

root = Tk()
root.title("Trivia")
root.geometry("1000x800")

#creating canvas
canvas = Canvas(root, width=800, height=600, bg="khaki1")
canvas.place(relx=.5, rely=.5,anchor= CENTER) #anchors the canvas in the center

#adding text to the canvas
game_display = canvas.create_text(200, 75, text="", fill="black", font=('Helvetica 15 bold'))

#creating the label that will display if something is correct/incorrect
feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

#creating the label to track the points
points = Label(root, text="", pady=10, font=("arial", 15, "bold"))


user_answer = StringVar()
user_answer.set("")

score = 0
corr_q = 0

def generate_qlist(json):
    question_list = []
    for i in json:
        i = i["question"]
        question_list.append(i)
    global total_q
    total_q = len(question_list)
    return question_list
    

def gen_question(list_):
    choice = random.choice(list_)
    return choice
    list_.remove(choice)    

def generate_answers(json, question): 
    option_list = []
    for i in get_request:
        if i["question"] == question:
            cor = i["correctAnswer"]
            option_list.append(cor)
            for incor in i["incorrectAnswers"]:
                option_list.append(incor)
    random.shuffle(option_list)
    return option_list

def ask_question(question, answers):
    canvas.itemconfig(game_display, text=question, width=300)
    
def gen_radiobuttons(answers):
    global frame
    frame = Frame(canvas, width =500)
    frame.place(x=50, y=200)
    y_pos = 150
    for i in answers:
        rad_btn = Radiobutton(frame, text=i, variable=user_answer, command=eval_score, value=i, font=('Helvetica 10 bold'))
        rad_btn.pack(anchor = "w")
        y_pos+=20
    
def eval_score():
    answer = user_answer.get()
    for i in get_request:
        if i["question"] == current_question:
            if i["correctAnswer"] == answer:
                feedback["text"] = "Correct! Next Question"
                feedback.place(x=50, y=10)
                global score
                score += 10
                global corr_q
                corr_q += 1
            else:
                feedback["text"] = "incorrect :("
                feedback.place(x=50, y=10)
        points["text"] = f"Points: {score}"
        points.place(x=50, y=50)

def main(): 
#     global q_list
#     q_list = generate_qlist(get_request)
#     print (q_list)
    global current_question
    current_question = gen_question(q_list)
    q_list.remove(current_question)
    ans_list = generate_answers(get_request, current_question)
    ask_question(current_question, ans_list)
    gen_radiobuttons(ans_list)
    

#get new question
def get_new():
    if len(q_list) == 0:
        feedback["text"] = f"Done! You got {corr_q} out {total_q} questions right!"
        canvas.delete("all")
    else: 
        # clears the frame (source: https://www.tutorialspoint.com/how-to-clear-out-a-frame-in-the-tkinter)
        for widgets in frame.winfo_children():
            widgets.destroy()
        main()
        
    

q_list = generate_qlist(get_request)
main()
new_quest = Button (canvas, text = "Next Question", pady=10, command = get_new, font=("arial", 15, "bold"))
new_quest.place(x=50, y=500)
root.mainloop()